# Intro to Table Detection with Fast RCNN

By taking an ImageNet-pretrained model such as the VGG16, we can add a few more convolutional layers to construct an RPN, or region proposal network. This module extracts regions of interest, or RoIs, that inform a model on where to identify an object. 


When the RoIs are applied, we do max pooling only in the regions of interest, as to find an embedding that uniquely identifies that area of the input and well as building a description of what object might lie in that region. From this description, the model can then categorize that region into one of k categories it was trained to recognize. 


In [1]:
# Train Fast RCNN

import logging
import pprint
import mxnet as mx
import numpy as np

from rcnn.config import config, default, generate_config
from rcnn.symbol import *
from rcnn.core import callback, metric
from rcnn.core.loader import AnchorLoader
from rcnn.core.module import MutableModule
from rcnn.utils.load_data import load_gt_roidb, merge_roidb, filter_roidb
from rcnn.utils.load_model import load_param


def train_net(args, ctx, pretrained, epoch, prefix, begin_epoch, end_epoch,
              lr=0.001, lr_step='5'):
    # set up logger
    logging.basicConfig()
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # setup config
    config.TRAIN.BATCH_IMAGES = 1
    config.TRAIN.BATCH_ROIS = 128
    config.TRAIN.END2END = True
    config.TRAIN.BBOX_NORMALIZATION_PRECOMPUTED = True

    # load symbol
    sym = eval('get_' + args.network + '_train')(num_classes=config.NUM_CLASSES, num_anchors=config.NUM_ANCHORS)
    feat_sym = sym.get_internals()['rpn_cls_score_output']


    # setup multi-gpu
    batch_size = len(ctx)
    input_batch_size = config.TRAIN.BATCH_IMAGES * batch_size

    # print config
    pprint.pprint(config)

    # load dataset and prepare imdb for training
    image_sets = [iset for iset in args.image_set.split('+')]
    roidbs = [load_gt_roidb(args.dataset, image_set, args.root_path, args.dataset_path,
                            flip=not args.no_flip)
              for image_set in image_sets]
    roidb = merge_roidb(roidbs)
    roidb = filter_roidb(roidb)

    # load training data
    train_data = AnchorLoader(feat_sym, roidb, batch_size=input_batch_size, shuffle=not args.no_shuffle,
                              ctx=ctx, work_load_list=args.work_load_list,
                              feat_stride=config.RPN_FEAT_STRIDE, anchor_scales=config.ANCHOR_SCALES,
                              anchor_ratios=config.ANCHOR_RATIOS, aspect_grouping=config.TRAIN.ASPECT_GROUPING)

    # infer max shape
    max_data_shape = [('data', (input_batch_size, 3, max([v[0] for v in config.SCALES]), max([v[1] for v in config.SCALES])))]
    max_data_shape, max_label_shape = train_data.infer_shape(max_data_shape)
    max_data_shape.append(('gt_boxes', (input_batch_size, 100, 5)))
    print('providing maximum shape', max_data_shape, max_label_shape)

    # infer shape
    data_shape_dict = dict(train_data.provide_data + train_data.provide_label)
    arg_shape, out_shape, aux_shape = sym.infer_shape(**data_shape_dict)
    arg_shape_dict = dict(zip(sym.list_arguments(), arg_shape))
    out_shape_dict = dict(zip(sym.list_outputs(), out_shape))
    aux_shape_dict = dict(zip(sym.list_auxiliary_states(), aux_shape))
    print('output shape')
    pprint.pprint(out_shape_dict)

    # load and initialize params
    if args.resume:
        arg_params, aux_params = load_param(prefix, begin_epoch, convert=True)
    else:
        arg_params, aux_params = load_param(pretrained, epoch, convert=True)
        arg_params['rpn_conv_3x3_weight'] = mx.random.normal(0, 0.01, shape=arg_shape_dict['rpn_conv_3x3_weight'])
        arg_params['rpn_conv_3x3_bias'] = mx.nd.zeros(shape=arg_shape_dict['rpn_conv_3x3_bias'])
        arg_params['rpn_cls_score_weight'] = mx.random.normal(0, 0.01, shape=arg_shape_dict['rpn_cls_score_weight'])
        arg_params['rpn_cls_score_bias'] = mx.nd.zeros(shape=arg_shape_dict['rpn_cls_score_bias'])
        arg_params['rpn_bbox_pred_weight'] = mx.random.normal(0, 0.01, shape=arg_shape_dict['rpn_bbox_pred_weight'])
        arg_params['rpn_bbox_pred_bias'] = mx.nd.zeros(shape=arg_shape_dict['rpn_bbox_pred_bias'])
        arg_params['cls_score_weight'] = mx.random.normal(0, 0.01, shape=arg_shape_dict['cls_score_weight'])
        arg_params['cls_score_bias'] = mx.nd.zeros(shape=arg_shape_dict['cls_score_bias'])
        arg_params['bbox_pred_weight'] = mx.random.normal(0, 0.001, shape=arg_shape_dict['bbox_pred_weight'])
        arg_params['bbox_pred_bias'] = mx.nd.zeros(shape=arg_shape_dict['bbox_pred_bias'])

    # check parameter shapes
    for k in sym.list_arguments():
        if k in data_shape_dict:
            continue
        assert k in arg_params, k + ' not initialized'
        assert arg_params[k].shape == arg_shape_dict[k], \
            'shape inconsistent for ' + k + ' inferred ' + str(arg_shape_dict[k]) + ' provided ' + str(arg_params[k].shape)
    for k in sym.list_auxiliary_states():
        assert k in aux_params, k + ' not initialized'
        assert aux_params[k].shape == aux_shape_dict[k], \
            'shape inconsistent for ' + k + ' inferred ' + str(aux_shape_dict[k]) + ' provided ' + str(aux_params[k].shape)

    # create solver
    fixed_param_prefix = config.FIXED_PARAMS
    data_names = [k[0] for k in train_data.provide_data]
    label_names = [k[0] for k in train_data.provide_label]
    mod = MutableModule(sym, data_names=data_names, label_names=label_names,
                        logger=logger, context=ctx, work_load_list=args.work_load_list,
                        max_data_shapes=max_data_shape, max_label_shapes=max_label_shape,
                        fixed_param_prefix=fixed_param_prefix)

    # decide training params
    # metric
    rpn_eval_metric = metric.RPNAccMetric()
    rpn_cls_metric = metric.RPNLogLossMetric()
    rpn_bbox_metric = metric.RPNL1LossMetric()
    eval_metric = metric.RCNNAccMetric()
    cls_metric = metric.RCNNLogLossMetric()
    bbox_metric = metric.RCNNL1LossMetric()
    eval_metrics = mx.metric.CompositeEvalMetric()
    for child_metric in [rpn_eval_metric, rpn_cls_metric, rpn_bbox_metric, eval_metric, cls_metric, bbox_metric]:
        eval_metrics.add(child_metric)
        
    # callback
    batch_end_callback = callback.Speedometer(train_data.batch_size, frequent=args.frequent)
    means = np.tile(np.array(config.TRAIN.BBOX_MEANS), config.NUM_CLASSES)
    stds = np.tile(np.array(config.TRAIN.BBOX_STDS), config.NUM_CLASSES)
    epoch_end_callback = callback.do_checkpoint(prefix, means, stds)
    
    # decide learning rate
    base_lr = lr
    lr_factor = 0.1
    lr_epoch = [int(epoch) for epoch in lr_step.split(',')]
    lr_epoch_diff = [epoch - begin_epoch for epoch in lr_epoch if epoch > begin_epoch]
    lr = base_lr * (lr_factor ** (len(lr_epoch) - len(lr_epoch_diff)))
    lr_iters = [int(epoch * len(roidb) / batch_size) for epoch in lr_epoch_diff]
    print('lr', lr, 'lr_epoch_diff', lr_epoch_diff, 'lr_iters', lr_iters)
    lr_scheduler = mx.lr_scheduler.MultiFactorScheduler(lr_iters, lr_factor)
    # optimizer
    optimizer_params = {'momentum': 0.9,
                        'wd': 0.0005,
                        'learning_rate': lr,
                        'lr_scheduler': lr_scheduler,
                        'rescale_grad': (1.0 / batch_size),
                        'clip_gradient': 5}

    # train
    mod.fit(train_data, eval_metric=eval_metrics, epoch_end_callback=epoch_end_callback,
            batch_end_callback=batch_end_callback, kvstore=args.kvstore,
            optimizer='sgd', optimizer_params=optimizer_params,
            arg_params=arg_params, aux_params=aux_params, 
            begin_epoch=begin_epoch, num_epoch=end_epoch)



In [ ]:
## Training Args
class DictToObject:
    '''
    helper class to encapsulate all the args from dict to obj
    '''
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = {'lr': 0.001, 'image_set': '2007_trainval', 'network': 'vgg',
        'resume': False, 'pretrained': 'model/vgg16', 'root_path': 'data',
        'dataset': 'PascalVOC', 'lr_step': '7', 'prefix': 'model/orge2e', 
        'end_epoch': 10, 'dataset_path': 'data/VOCdevkit', 
        'gpus': '0',
        'no_flip': False, 'no_shuffle': False, 'begin_epoch': 0, 
        'work_load_list': None, 'pretrained_epoch': 0,
        'kvstore': 'device', 'frequent': 20}

args = DictToObject(**args)
if len(args.gpus) > 1:
    ctx = [mx.gpu(int(i)) for i in args.gpus.split(',')]
else:
    ctx = [mx.gpu(int(args.gpus))]
train_net(args, ctx, args.pretrained, args.pretrained_epoch, args.prefix, args.begin_epoch, args.end_epoch,
              lr=args.lr, lr_step=args.lr_step)


INFO:root:voc_2007_trainval num_images 5011
INFO:root:voc_2007_trainval gt roidb loaded from data/cache/voc_2007_trainval_gt_roidb.pkl
INFO:root:voc_2007_trainval append flipped images to roidb


{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [8, 16, 32],
 'FIXED_PARAMS': ['conv1', 'conv2'],
 'FIXED_PARAMS_SHARED': ['conv1', 'conv2', 'conv3', 'conv4', 'conv5'],
 'IMAGE_STRIDE': 0,
 'NUM_ANCHORS': 9,
 'NUM_CLASSES': 21,
 'PIXEL_MEANS': array([ 103.939,  116.779,  123.68 ]),
 'RCNN_FEAT_STRIDE': 16,
 'RPN_FEAT_STRIDE': 16,
 'SCALES': [(600, 1000)],
 'TEST': {'BATCH_IMAGES': 1,
          'CXX_PROPOSAL': True,
          'HAS_RPN': False,
          'NMS': 0.3,
          'PROPOSAL_MIN_SIZE': 16,
          'PROPOSAL_NMS_THRESH': 0.7,
          'PROPOSAL_POST_NMS_TOP_N': 2000,
          'PROPOSAL_PRE_NMS_TOP_N': 20000,
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000},
 'TRAIN': {'ASPECT_GROUPING': True,
           'BATCH_IMAGES': 1,
           'BATCH_ROIS': 128,
           'BBOX_MEANS': [0.0, 0.0, 0.0, 0.0],
           'BBOX_NORMALIZATION_PRECOMPUTED': True,
           'BBOX_REGRESSION_THRESH': 0.

[393 498]
data/VOCdevkit/VOC2007/JPEGImages/000141.jpg
[[ 82  52 475 398]]
[82]
[475]
data/VOCdevkit/VOC2007/JPEGImages/000142.jpg
[[ 35  80 474 265]]
[35]
[474]
data/VOCdevkit/VOC2007/JPEGImages/000143.jpg
[[  0  45 330 332]]
[0]
[330]
data/VOCdevkit/VOC2007/JPEGImages/000146.jpg
[[122 207 219 407]]
[122]
[219]
data/VOCdevkit/VOC2007/JPEGImages/000147.jpg
[[435 104 467 145]
 [386 113 423 148]
 [325 124 354 149]
 [ 98 120 127 152]
 [ 35 111  65 148]]
[435 386 325  98  35]
[467 423 354 127  65]
data/VOCdevkit/VOC2007/JPEGImages/000150.jpg
[[181  77 314 263]
 [121  53 147 107]
 [ 64  50  85 108]]
[181 121  64]
[314 147  85]
data/VOCdevkit/VOC2007/JPEGImages/000153.jpg
[[142 146 263 190]]
[142]
[263]
data/VOCdevkit/VOC2007/JPEGImages/000154.jpg
[[133  75 441 265]]
[133]
[441]
data/VOCdevkit/VOC2007/JPEGImages/000156.jpg
[[119  72 459 351]
 [  4 102  68 143]]
[119   4]
[459  68]
data/VOCdevkit/VOC2007/JPEGImages/000158.jpg
[[233  82 304 206]]
[233]
[304]
data/VOCdevkit/VOC2007/JPEGImages/0

data/VOCdevkit/VOC2007/JPEGImages/000411.jpg
[[ 82 189 113 276]
 [  7 193  41 274]
 [ 55 195  73 235]
 [121 192 152 293]
 [185 194 220 296]
 [150 184 175 292]]
[ 82   7  55 121 185 150]
[113  41  73 152 220 175]
data/VOCdevkit/VOC2007/JPEGImages/000416.jpg
[[101 138 170 247]
 [241 170 319 251]
 [155 153 283 243]]
[101 241 155]
[170 319 283]
data/VOCdevkit/VOC2007/JPEGImages/000417.jpg
[[131  81 326 374]]
[131]
[326]
data/VOCdevkit/VOC2007/JPEGImages/000419.jpg
[[ 14  38 477 321]
 [459  15 498 220]
 [391   0 439  61]]
[ 14 459 391]
[477 498 439]
data/VOCdevkit/VOC2007/JPEGImages/000420.jpg
[[ 25  28 422 346]]
[25]
[422]
data/VOCdevkit/VOC2007/JPEGImages/000424.jpg
[[366 158 435 215]
 [216 196 311 253]
 [ 99 208 173 251]
 [ 14 176  89 226]]
[366 216  99  14]
[435 311 173  89]
data/VOCdevkit/VOC2007/JPEGImages/000427.jpg
[[114  23 461 274]]
[114]
[461]
data/VOCdevkit/VOC2007/JPEGImages/000428.jpg
[[124 105 424 403]
 [  0 322 296 411]
 [149 179 479 413]]
[124   0 149]
[424 296 479]
data/VO

[276 226   1]
[482 347 430]
data/VOCdevkit/VOC2007/JPEGImages/000728.jpg
[[  0  38 413 330]]
[0]
[413]
data/VOCdevkit/VOC2007/JPEGImages/000729.jpg
[[  3   5 365 497]]
[3]
[365]
data/VOCdevkit/VOC2007/JPEGImages/000730.jpg
[[  5  51 495 233]]
[5]
[495]
data/VOCdevkit/VOC2007/JPEGImages/000731.jpg
[[ 92 386 191 499]
 [ 29  74 316 372]
 [164   0 328 272]]
[ 92  29 164]
[191 316 328]
data/VOCdevkit/VOC2007/JPEGImages/000733.jpg
[[177  24 402 382]
 [  2 200 347 434]]
[177   2]
[402 347]
data/VOCdevkit/VOC2007/JPEGImages/000738.jpg
[[  4 123 499 234]]
[4]
[499]
data/VOCdevkit/VOC2007/JPEGImages/000739.jpg
[[  0  29  96 373]
 [ 21  82 114 373]
 [ 85  64 228 373]
 [165  54 261 373]
 [248  54 335 373]
 [307  65 403 373]
 [392  25 497 373]]
[  0  21  85 165 248 307 392]
[ 96 114 228 261 335 403 497]
data/VOCdevkit/VOC2007/JPEGImages/000740.jpg
[[ 96  84 210 275]
 [288 103 396 270]]
[ 96 288]
[210 396]
data/VOCdevkit/VOC2007/JPEGImages/000742.jpg
[[ 88 128 109 167]
 [189 144 499 334]
 [204  98 2

...

[[ 19  18 324 393]]
[19]
[324]
data/VOCdevkit/VOC2007/JPEGImages/009382.jpg
[[  0 235 332 499]
 [ 28  74 299 484]
 [184   0 322 369]]
[  0  28 184]
[332 299 322]
data/VOCdevkit/VOC2007/JPEGImages/009386.jpg
[[ 91   1 497 373]]
[91]
[497]
data/VOCdevkit/VOC2007/JPEGImages/009388.jpg
[[114 172 186 232]
 [180 152 241 218]
 [ 83 264 266 374]
 [337 176 400 251]
 [427 227 446 278]]
[114 180  83 337 427]
[186 241 266 400 446]
data/VOCdevkit/VOC2007/JPEGImages/009389.jpg
[[  0 116 381 363]]
[0]
[381]
data/VOCdevkit/VOC2007/JPEGImages/009392.jpg
[[  0 193  90 330]
 [206 229 251 246]
 [251 222 285 236]
 [294 222 327 236]
 [186 222 210 232]
 [449 223 481 237]
 [426 282 499 330]]
[  0 206 251 294 186 449 426]
[ 90 251 285 327 210 481 499]
data/VOCdevkit/VOC2007/JPEGImages/009393.jpg
[[179 139 355 341]
 [ 21 159 213 371]
 [ 43   1 205 371]
 [398 161 462 295]]
[179  21  43 398]
[355 213 205 462]
data/VOCdevkit/VOC2007/JPEGImages/009394.jpg
[[  0   1 497 253]
 [341 159 367 260]]
[  0 341]
[497 367]
d

INFO:root:load data: filtered 0 roidb entries: 10022 -> 10022



[0]
[457]
data/VOCdevkit/VOC2007/JPEGImages/009711.jpg
[[ 72  58 463 313]
 [  0 149  78 220]
 [146 185 185 302]
 [ 81 205 123 303]]
[ 72   0 146  81]
[463  78 185 123]
data/VOCdevkit/VOC2007/JPEGImages/009712.jpg
[[257 167 313 374]
 [205 179 261 374]]
[257 205]
[313 261]
data/VOCdevkit/VOC2007/JPEGImages/009713.jpg
[[ 94  56 386 343]
 [265   0 368 106]
 [  3   3 142 175]]
[ 94 265   3]
[386 368 142]
data/VOCdevkit/VOC2007/JPEGImages/009717.jpg
[[382 153 498 235]
 [ 75 205 237 332]
 [230 162 318 282]
 [304 177 365 233]
 [ 42  74 160 177]
 [162  76 262 158]
 [269  77 353 149]
 [340 106 385 148]
 [445 102 498 136]
 [  0 101  53 122]
 [372 181 410 284]]
[382  75 230 304  42 162 269 340 445   0 372]
[498 237 318 365 160 262 353 385 498  53 410]
data/VOCdevkit/VOC2007/JPEGImages/009718.jpg
[[203 193 498 251]
 [ 92 314 138 343]
 [260 336 338 371]]
[203  92 260]
[498 138 338]
data/VOCdevkit/VOC2007/JPEGImages/009719.jpg
[[  0 105 341 412]
 [ 81  69 494 411]]
[ 0 81]
[341 494]
data/VOCdevkit/V

('providing maximum shape', [('data', (1, 3, 600, 1000)), ('gt_boxes', (1, 100, 5))], [('label', (1, 20646)), ('bbox_target', (1, 36, 37, 62)), ('bbox_weight', (1, 36, 37, 62))])
output shape
{'bbox_loss_reshape_output': (1L, 128L, 84L),
 'blockgrad0_output': (1L, 128L),
 'cls_prob_reshape_output': (1L, 128L, 21L),
 'rpn_bbox_loss_output': (1L, 36L, 32L, 62L),
 'rpn_cls_prob_output': (1L, 2L, 288L, 62L)}
model/vgg16
vgg16
('lr', 0.001, 'lr_epoch_diff', [7], 'lr_iters', [70154])


INFO:root:Epoch[0] Batch [20]	Speed: 0.42 samples/sec	Train-RPNAcc=0.908854,	RPNLogLoss=0.303647,	RPNL1Loss=1.331508,	RCNNAcc=0.731027,	RCNNLogLoss=1.396968,	RCNNL1Loss=2.578552,	
INFO:root:Epoch[0] Batch [40]	Speed: 0.71 samples/sec	Train-RPNAcc=0.926258,	RPNLogLoss=0.256415,	RPNL1Loss=1.141757,	RCNNAcc=0.770579,	RCNNLogLoss=1.146583,	RCNNL1Loss=2.595929,	
INFO:root:Epoch[0] Batch [60]	Speed: 0.55 samples/sec	Train-RPNAcc=0.937564,	RPNLogLoss=0.221250,	RPNL1Loss=1.101883,	RCNNAcc=0.794314,	RCNNLogLoss=1.020761,	RCNNL1Loss=2.520255,	
INFO:root:Epoch[0] Batch [80]	Speed: 0.57 samples/sec	Train-RPNAcc=0.938850,	RPNLogLoss=0.202296,	RPNL1Loss=1.047270,	RCNNAcc=0.798322,	RCNNLogLoss=0.976983,	RCNNL1Loss=2.520876,	
INFO:root:Epoch[0] Batch [100]	Speed: 1.13 samples/sec	Train-RPNAcc=0.946202,	RPNLogLoss=0.188144,	RPNL1Loss=0.983935,	RCNNAcc=0.801361,	RCNNLogLoss=0.935375,	RCNNL1Loss=2.535259,	
INFO:root:Epoch[0] Batch [120]	Speed: 0.67 samples/sec	Train-RPNAcc=0.948993,	RPNLogLoss=0.180534,	

In [ ]:
# Fast r-cnn trained on VOC2007 dataset

import os
import cv2
import mxnet as mx
import numpy as np
from rcnn.config import config
from rcnn.symbol import get_vgg_test, get_vgg_rpn_test
from rcnn.io.image import resize, transform
from rcnn.core.tester import Predictor, im_detect, im_proposal, vis_all_detection, draw_all_detection
from rcnn.utils.load_model import load_param
from rcnn.processing.nms import py_nms_wrapper, cpu_nms_wrapper, gpu_nms_wrapper

import urllib2
import tempfile

# 20 classes
CLASSES = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')

config.TEST.HAS_RPN = True
SHORT_SIDE = config.SCALES[0][0]
LONG_SIDE = config.SCALES[0][1]
PIXEL_MEANS = config.PIXEL_MEANS
DATA_NAMES = ['data', 'im_info']
LABEL_NAMES = None
DATA_SHAPES = [('data', (1, 3, LONG_SIDE, SHORT_SIDE)), ('im_info', (1, 3))]
LABEL_SHAPES = None

# visualization
CONF_THRESH = 0.7
NMS_THRESH = 0.3
nms = py_nms_wrapper(NMS_THRESH)


def get_net(symbol, prefix, epoch, ctx):
    arg_params, aux_params = load_param(prefix, epoch, convert=True, ctx=ctx, process=True)

    # infer shape
    data_shape_dict = dict(DATA_SHAPES)
    arg_names, aux_names = symbol.list_arguments(), symbol.list_auxiliary_states()
    arg_shape, _, aux_shape = symbol.infer_shape(**data_shape_dict)
    arg_shape_dict = dict(zip(arg_names, arg_shape))
    aux_shape_dict = dict(zip(aux_names, aux_shape))

    # check shapes
    for k in symbol.list_arguments():
        if k in data_shape_dict or 'label' in k:
            continue
        assert k in arg_params, k + ' not initialized'
        assert arg_params[k].shape == arg_shape_dict[k], \
            'shape inconsistent for ' + k + ' inferred ' + str(arg_shape_dict[k]) + ' provided ' + str(arg_params[k].shape)
    for k in symbol.list_auxiliary_states():
        assert k in aux_params, k + ' not initialized'
        assert aux_params[k].shape == aux_shape_dict[k], \
            'shape inconsistent for ' + k + ' inferred ' + str(aux_shape_dict[k]) + ' provided ' + str(aux_params[k].shape)

    predictor = Predictor(symbol, DATA_NAMES, LABEL_NAMES, context=ctx,
                          provide_data=DATA_SHAPES, provide_label=LABEL_SHAPES,
                          arg_params=arg_params, aux_params=aux_params)
    return predictor


def generate_batch(im):
    """
    preprocess image, return batch
    :param im: cv2.imread returns [height, width, channel] in BGR
    :return:
    data_batch: MXNet input batch
    data_names: names in data_batch
    im_scale: float number
    """
    im_array, im_scale = resize(im, SHORT_SIDE, LONG_SIDE)
    im_array = transform(im_array, PIXEL_MEANS)
    im_info = np.array([[im_array.shape[2], im_array.shape[3], im_scale]], dtype=np.float32)
    data = [mx.nd.array(im_array), mx.nd.array(im_info)]
    data_shapes = [('data', im_array.shape), ('im_info', im_info.shape)]
    data_batch = mx.io.DataBatch(data=data, label=None, provide_data=data_shapes, provide_label=None)
    return data_batch, DATA_NAMES, im_scale


def demo_net(predictor, im, vis=False):
    """
    generate data_batch -> im_detect -> post process
    :param predictor: Predictor
    :param image_name: image name
    :param vis: will save as a new image if not visualized
    :return: None
    """

    data_batch, data_names, im_scale = generate_batch(im)
    scores, boxes, data_dict = im_detect(predictor, data_batch, data_names, im_scale)

    all_boxes = [[] for _ in CLASSES]
    for cls in CLASSES:
        cls_ind = CLASSES.index(cls)
        cls_boxes = boxes[:, 4 * cls_ind:4 * (cls_ind + 1)]
        cls_scores = scores[:, cls_ind, np.newaxis]
        keep = np.where(cls_scores >= CONF_THRESH)[0]
        dets = np.hstack((cls_boxes, cls_scores)).astype(np.float32)[keep, :]
        keep = nms(dets)
        all_boxes[cls_ind] = dets[keep, :]

    boxes_this_image = [[]] + [all_boxes[j] for j in range(1, len(CLASSES))]

    # print results
    print('class ---- [[x1, x2, y1, y2, confidence]]')
    for ind, boxes in enumerate(boxes_this_image):
        if len(boxes) > 0:
            print('---------', CLASSES[ind], '---------')
            print(boxes)

    if vis:
        vis_all_detection(data_dict['data'].asnumpy(), boxes_this_image, CLASSES, im_scale)
    else:
        #result_file = image_name.replace('.', '_result.')
        result_file = "output.jpg"
        print('results saved to %s' % result_file)
        im = draw_all_detection(data_dict['data'].asnumpy(), boxes_this_image, CLASSES, im_scale)
        cv2.imwrite(result_file, im)

def get_image_from_url(url, img_file):


    req = urllib2.urlopen(url)
    img_file.write(req.read())
    img_file.flush()
    return img_file.name

## Inference - Lets run some predictions


In [ ]:
vis = True
gpu = 0
epoch = 1
prefix = 'e2e'

ctx = mx.gpu(gpu)
symbol = get_vgg_test(num_classes=config.NUM_CLASSES, num_anchors=config.NUM_ANCHORS)
predictor = get_net(symbol, prefix, epoch, ctx)



In [ ]:
img_file = tempfile.NamedTemporaryFile()
#url = 'http://images.all-free-download.com/images/graphiclarge/aeroplane_boeing_737_air_new_zealand_218019.jpg'
url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2012/segexamples/images/21.jpg'
#url = 'https://www.siemens.com/press/pool/de/pressebilder/2011/mobility/soimo201107/072dpi/soimo201107-04_072dpi.jpg'

image = get_image_from_url(url, img_file)
assert os.path.exists(image), image + ' not found'
im = cv2.imread(image)
demo_net(predictor, im, vis)

## Live object detection through webcam

In [ ]:
from IPython.display import HTML
from StringIO import StringIO
import numpy as np
import cv2

vis = False
def classify(img):
    img = img[len('data:image/png;base64,'):].decode('base64')
    #img_stream = StringIO()
    #img_stream.write(img)
    #img_stream.seek(0)
    #img_array = np.asarray(bytearray(img_stream.read()), dtype=np.uint8)
    img_file = tempfile.NamedTemporaryFile()
    img_file.write(img)
    img_file.flush()
    img_array = cv2.imread(img_file.name)
    demo_net(predictor, img_array, vis)
    return "done"
    
HTML(data=''' 
<style>
#container {
    margin: 0px auto;
    width: 250px;
    height: 250px;
    border: 10px #333 solid;
}
#videoElement {
    width: 250px;
    height: 250px;
    background-color: #666;
}
</style>
<input type=button value="Capture Image" onClick="take_snapshot()">

<video autoplay="true" id="videoElement"></video>
<canvas id="c" style="display:none;" width="300" height="300"></canvas>
<canvas id="display" width="300" height="300"></canvas>


<script>

var video = document.querySelector("#videoElement");
navigator.getUserMedia = navigator.getUserMedia || navigator.webkitGetUserMedia || navigator.mozGetUserMedia || navigator.msGetUserMedia || navigator.oGetUserMedia;
 
if (navigator.getUserMedia) {       
    navigator.getUserMedia({video: true}, handleVideo, videoError);
}
 
function handleVideo(stream) {
    video.src = window.URL.createObjectURL(stream);
}
 
function videoError(e) {
    // do something
}


function handle_output(out) {
    console.log("done classifying", out);
    // display image
    var ctx = $("#display")[0].getContext("2d"),
    img = new Image();
    
    img.onload = function(){
        ctx.drawImage(img, 0, 0, 300, 300);
        $("span").text("Loaded.");
    };
    img.src = "output.jpg";
    
    //console.log("done classifying", out.content.data["text/plain"]);
}

function take_snapshot() {
    var canvas = document.getElementById('c');
    var kernel = IPython.notebook.kernel;
    canvas.getContext("2d").drawImage(video, 0, 0, 300, 300);

    data = canvas.toDataURL('image/png');
    console.log("captured data");
    kernel.execute("classify('" + data + "')", {
        'iopub': {
            'output': handle_output
        }
    }, {
        silent: false
    });
}
</script>

''')



## 